In [2]:
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import ta
from copy import deepcopy
import math
from datetime import date , datetime
from dateutil.tz import tzoffset
from backtesting import Strategy
from backtesting.lib import crossover
from backtesting import Backtest
from backtesting.lib import crossover
import backtesting
backtesting.set_bokeh_output(notebook=False)
import warnings
warnings.filterwarnings("ignore")
np.random.seed(12345)

In [3]:
stocks = ['AMD','AAPL','JPM','PEP','NFLX']
for stock in stocks:
    os.makedirs(f'data/{stock}', exist_ok=True) 
    data = yf.download(stock,start="2012-01-01", end="2022-01-01")
    data.to_csv(f'data/{stock}/{stock}.csv')

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [4]:
amd = pd.read_csv('data/AMD/AMD.csv',index_col='Date', parse_dates=True).astype(float)
aapl = pd.read_csv('data/AAPL/AAPL.csv',index_col='Date', parse_dates=True).astype(float)
jpm = pd.read_csv('data/JPM/JPM.csv',index_col='Date', parse_dates=True).astype(float)
pep = pd.read_csv('data/PEP/PEP.csv',index_col='Date', parse_dates=True).astype(float)
nflx = pd.read_csv('data/NFLX/NFLX.csv',index_col='Date', parse_dates=True).astype(float)

In [5]:
amd_train = amd[amd.index < datetime(2021, 1, 1)]
amd_test = amd[amd.index >= datetime(2021, 1, 1)]
aapl_train = aapl[aapl.index < datetime(2021, 1, 1)]
aapl_test = aapl[aapl.index >= datetime(2021, 1, 1)]
jpm_train = jpm[jpm.index < datetime(2021, 1, 1)]
jpm_test = jpm[jpm.index >= datetime(2021, 1, 1)]
pep_train = pep[pep.index < datetime(2021, 1, 1)]
pep_test = pep[pep.index >= datetime(2021, 1, 1)]
nflx_train = nflx[nflx.index < datetime(2021, 1, 1)]
nflx_test = nflx[nflx.index >= datetime(2021, 1, 1)]
trains = [amd_train,aapl_train,jpm_train,pep_train,nflx_train]
tests = [amd_test,aapl_test,jpm_test,pep_test,nflx_test]

In [6]:
for df in trains:
    multiplier = df['Adj Close'] / df['Close']
    df['Close'] = df['Adj Close']
    df['High'] = multiplier * df['High']
    df['Low'] = multiplier * df['Low']
    df['Open'] = multiplier * df['Open']
    df.drop(['Adj Close','Volume'],axis=1,inplace=True)

In [7]:
for df in tests:
    multiplier = df['Adj Close'] / df['Close']
    df['Close'] = df['Adj Close']
    df['High'] = multiplier * df['High']
    df['Low'] = multiplier * df['Low']
    df['Open'] = multiplier * df['Open']
    df.drop(['Adj Close','Volume'],axis=1,inplace=True)    

## Most Common RSI Strategy for Benchmark

In [8]:
def EMA(close,period):
    return ta.trend.ema_indicator(close,window=period,fillna=True)
def RSI(close,period):
    return ta.momentum.RSIIndicator(close,window=period,fillna=True).rsi()
class RSIStrategy(Strategy):
    period = 0
    buy_value = 0
    sell_value = 0
    def init(self):
        p = int(round(self.period))
        self.rsi = self.I(RSI,pd.Series(self.data['Close']), p)
    def next(self):
        if crossover(self.rsi,self.buy_value):
            self.buy()
        elif crossover(self.sell_value,self.rsi):
            self.position.close()

In [9]:
train_stats = pd.DataFrame(columns=['Sharpe Ratio', 'Annual Return'])
test_stats = pd.DataFrame(columns=['Sharpe Ratio', 'Annual Return'])

In [10]:
for index,stock in enumerate(stocks):
    bt = Backtest(trains[index], RSIStrategy, cash=10_000, commission=0)
    stats = bt.run(period=14,buy_value=30,sell_value=70)
    train_stats.loc[stock] = {'Sharpe Ratio': stats['Sharpe Ratio'], 'Annual Return': stats['Return (Ann.) [%]']}

In [11]:
for index,stock in enumerate(stocks):
    bt = Backtest(tests[index], RSIStrategy, cash=10_000, commission=0)
    stats = bt.run(period=14,buy_value=30,sell_value=70)
    test_stats.loc[stock] = {'Sharpe Ratio': stats['Sharpe Ratio'], 'Annual Return': stats['Return (Ann.) [%]']}

In [12]:
train_stats.loc['Mean'] = {'Sharpe Ratio':train_stats['Sharpe Ratio'].mean(),'Annual Return':train_stats['Annual Return'].mean()}
train_stats

,Sharpe Ratio,Annual Return
AMD,0.193693,7.582056
AAPL,0.240033,5.029479
JPM,0.441521,9.520845
PEP,0.385615,5.641887
NFLX,0.480074,13.577432
Mean,0.348187,8.270340


In [13]:
test_stats.loc['Mean'] = {'Sharpe Ratio':test_stats['Sharpe Ratio'].mean(),'Annual Return':test_stats['Annual Return'].mean()}
test_stats

,Sharpe Ratio,Annual Return
AMD,0.291812,6.724299
AAPL,0.746924,14.661221
JPM,0.405494,6.380710
PEP,0.034293,0.300075
NFLX,0.593072,11.240401
Mean,0.414319,7.861341


## GWO RSI Strategy Without Trend Specific Parameters

In [14]:
def check_constraints(params):
    if params[0] < 3 or params[0] > 30:
        return False
    if params[1] < 0 or params[1] > params[2] or params[1] > 100:
        return False
    if params[2] < 0 or params[2] > 100:
        return False
    return True

In [15]:
train_stats = pd.DataFrame(columns=['Sharpe Ratio', 'Annual Return'])
test_stats = pd.DataFrame(columns=['Sharpe Ratio', 'Annual Return'])
best_params = pd.DataFrame(columns=['Period', 'Buy Value','Sell Value'])

In [16]:
for index,stock in enumerate(stocks):
    bt = Backtest(trains[index], RSIStrategy, cash=10_000, commission=0)
    population_size = 25
    iterations = 30
    population=np.array([[1,2,3]])
    for j in range(population_size):
        random_params = np.random.uniform(10,90,size=(2))
        while random_params[0] > random_params[1]:
            random_params = np.random.uniform(10,90,size=(2))
        l = np.append(np.random.randint(3,30),random_params)
        population = np.vstack([population,l])
    population = population[1:]
    scores = np.array([])
    for wolf in population:
        stats = bt.run(period=wolf[0],buy_value=wolf[1],sell_value=wolf[2])
        scores = np.append(scores,stats['Sharpe Ratio'])
    scores = np.nan_to_num(scores,0)
    for i in range(1,iterations):
        alpha_wolf = population[np.where(scores == sorted(scores)[-1])[0][-1]]
        beta_wolf = population[np.where(scores == sorted(scores)[-2])[0][-1]]
        delta_wolf = population[np.where(scores == sorted(scores)[-3])[0][-1]]
        bests = [alpha_wolf , beta_wolf , delta_wolf]
        alpha = 2 * (1 - (i/iterations))
        for index,wolf in enumerate(population):
            rand = round(np.random.uniform(0,1),2)
            A = 2*alpha*rand-alpha
            rand = round(np.random.uniform(0,1),2)
            C = 2*rand
            Xs = np.array([1,2,3])
            for j in range(len(bests)):
                D = C*bests[j]-wolf
                new_xs = bests[j]-A*D
                Xs = np.vstack([Xs,new_xs])
            Xs=Xs[1:]
            Xnew = np.zeros(3)
            for x in Xs:
                Xnew[0] += x[0]
                Xnew[1] += x[1]
                Xnew[2] += x[2]
            Xnew = Xnew / 3
            if not check_constraints(Xnew):
                continue
            s1 = scores[index]
            s2 = bt.run(period=Xnew[0],buy_value=Xnew[1],sell_value=Xnew[2])
            if math.isnan(s2['Sharpe Ratio']):
                continue
            if s2['Sharpe Ratio'] > s1:
                wolf[0] = Xnew[0]
                wolf[1] = Xnew[1]
                wolf[2] = Xnew[2]
                scores[index] = s2['Sharpe Ratio']
        print(f'Stock: {stock}, Iteration: {i}, Best Sharpe: {max(scores)}')
    best = population[np.where(scores == sorted(scores)[-1])[0][-1]]
    best_params.loc[stock] = {'Period':int(round(best[0])), 'Buy Value':best[1],'Sell Value':best[2]}
    stats = bt.run(period=best[0],buy_value=best[1],sell_value=best[2])
    train_stats.loc[stock] = {'Sharpe Ratio': stats['Sharpe Ratio'], 'Annual Return': stats['Return (Ann.) [%]']}

Stock: AMD, Iteration: 1, Best Sharpe: 0.6569510453767335
Stock: AMD, Iteration: 2, Best Sharpe: 0.6569510453767335
Stock: AMD, Iteration: 3, Best Sharpe: 0.6569510453767335
Stock: AMD, Iteration: 4, Best Sharpe: 0.6569510453767335
Stock: AMD, Iteration: 5, Best Sharpe: 0.6569510453767335
Stock: AMD, Iteration: 6, Best Sharpe: 0.6569510453767335
Stock: AMD, Iteration: 7, Best Sharpe: 0.6569510453767335
Stock: AMD, Iteration: 8, Best Sharpe: 0.701976211664099
Stock: AMD, Iteration: 9, Best Sharpe: 0.701976211664099
Stock: AMD, Iteration: 10, Best Sharpe: 0.701976211664099
Stock: AMD, Iteration: 11, Best Sharpe: 0.701976211664099
Stock: AMD, Iteration: 12, Best Sharpe: 0.701976211664099
Stock: AMD, Iteration: 13, Best Sharpe: 0.701976211664099
Stock: AMD, Iteration: 14, Best Sharpe: 0.701976211664099
Stock: AMD, Iteration: 15, Best Sharpe: 0.701976211664099
Stock: AMD, Iteration: 16, Best Sharpe: 0.701976211664099
Stock: AMD, Iteration: 17, Best Sharpe: 0.701976211664099
Stock: AMD, Iter

In [17]:
best_params

,Period,Buy Value,Sell Value
AMD,8,21.546596,26.410565
AAPL,24,55.297510,76.328768
JPM,14,43.949514,53.521238
PEP,15,61.469095,67.192261
NFLX,8,52.977482,68.547385


In [18]:
train_stats.loc['Mean'] = {'Sharpe Ratio':train_stats['Sharpe Ratio'].mean(),'Annual Return':train_stats['Annual Return'].mean()}
train_stats

,Sharpe Ratio,Annual Return
AMD,0.701976,51.590089
AAPL,0.958991,29.845511
JPM,1.105214,21.789793
PEP,1.092395,11.977624
NFLX,0.925994,59.312302
Mean,0.956914,34.903064


In [19]:
for index,row in best_params.iterrows():
    bt = Backtest(tests[stocks.index(index)], RSIStrategy, cash=10_000, commission=0)
    stats = bt.run(period=row['Period'],buy_value=row['Buy Value'],sell_value=row['Sell Value'])
    test_stats.loc[index] = {'Sharpe Ratio': stats['Sharpe Ratio'], 'Annual Return': stats['Return (Ann.) [%]']}
test_stats.loc['Mean'] = {'Sharpe Ratio':test_stats['Sharpe Ratio'].mean(),'Annual Return':test_stats['Annual Return'].mean()}
test_stats

,Sharpe Ratio,Annual Return
AMD,0.000000,-20.020000
AAPL,1.051346,35.114453
JPM,1.095991,18.460597
PEP,1.613397,20.509404
NFLX,0.000000,-1.102588
Mean,0.752147,10.592373


## GWO RSI Strategy With Trend Specific Parameters

In [20]:
def check_constraints(params):
    if params[0] < 40 or params[0] > 200:
        return False
    if params[1] < 3 or params[1] > 30:
        return False
    if params[2] < 10 or params[2] > params[3] or params[2] > 90:
        return False
    if params[3] < 10 or params[3] > 90:
        return False
    if params[4] < 10 or params[4] > params[5] or params[3] > 90:
        return False
    if params[5] < 10 or params[5] > 90:
        return False
    return True

In [21]:
class RSITrendStrategy(Strategy):
    ema_period = 0
    period = 0
    up_trend_buy_value = 0
    up_trend_sell_value = 0
    down_trend_buy_value = 0
    down_trend_sell_value = 0
    buyed_uptrend = False
    buyed_downtrend = False
    def init(self):
        p = int(round(self.period))
        p2 = int(round(self.ema_period))
        self.rsi = self.I(RSI,pd.Series(self.data['Close']), p)
        self.ema = self.I(EMA,pd.Series(self.data['Close']), p2)  
    def next(self):
        if self.buyed_uptrend and crossover(self.up_trend_sell_value,self.rsi):
            self.position.close()
            self.buyed_uptrend = False
        if self.buyed_downtrend and crossover(self.down_trend_sell_value,self.rsi):
            self.position.close()
            self.buyed_downtrend = False
        if self.data['Close'][-1] > self.ema[-1]:
            #uptrend
            if crossover(self.rsi,self.up_trend_buy_value):
                self.buy()
                self.buyed_uptrend = True
        else:
            if crossover(self.rsi,self.down_trend_buy_value):
                self.buy()
                self.buyed_downtrend = True

In [22]:
train_stats = pd.DataFrame(columns=['Sharpe Ratio', 'Annual Return'])
test_stats = pd.DataFrame(columns=['Sharpe Ratio', 'Annual Return'])
best_params = pd.DataFrame(columns=['Ema Period','Period', 'Up Trend Buy Value','Up Trend Sell Value', 'Down Trend Buy Value','Down Trend Sell Value'])

In [23]:
for index,stock in enumerate(stocks):
    bt = Backtest(trains[index], RSITrendStrategy, cash=10_000, commission=0)
    population_size = 50
    iterations = 30
    population=np.array([[0,1,2,3,4,5]])
    for j in range(population_size):
        random_params = np.random.uniform(10,90,size=(4))
        while random_params[0] > random_params[1] or random_params[2] > random_params[3]:
            random_params = np.random.uniform(10,90,size=(4))
        l = np.append(np.random.randint(40,200),np.random.randint(3,30))
        l = np.append(l,random_params)
        population = np.vstack([population,l])
    population = population[1:]
    scores = np.array([])
    for wolf in population:
        stats = bt.run(
                ema_period=wolf[0], period = wolf[1],
                   up_trend_buy_value=wolf[2],up_trend_sell_value=wolf[3],
                   down_trend_buy_value=wolf[4],down_trend_sell_value=wolf[5],
                  )
        scores = np.append(scores,stats['Sharpe Ratio'])
    scores = np.nan_to_num(scores,0)
    for i in range(1,iterations):
        alpha_wolf = population[np.where(scores == sorted(scores)[-1])[0][-1]]
        beta_wolf = population[np.where(scores == sorted(scores)[-2])[0][-1]]
        delta_wolf = population[np.where(scores == sorted(scores)[-3])[0][-1]]
        bests = [alpha_wolf , beta_wolf , delta_wolf]
        alpha = 2 * (1 - (i/iterations))
        for index,wolf in enumerate(population):
            rand = round(np.random.uniform(0,1),2)
            A = 2*alpha*rand-alpha
            rand = round(np.random.uniform(0,1),2)
            C = 2*rand
            Xs = np.array([0,1,2,3,4,5])
            for j in range(len(bests)):
                D = C*bests[j]-wolf
                new_xs = bests[j]-A*D
                Xs = np.vstack([Xs,new_xs])
            Xs=Xs[1:]
            Xnew = np.zeros(6)
            for x in Xs:
                Xnew[0] += x[0]
                Xnew[1] += x[1]
                Xnew[2] += x[2]
                Xnew[3] += x[3]
                Xnew[4] += x[4]
                Xnew[5] += x[5]
            Xnew = Xnew / len(bests)
            if not check_constraints(Xnew):
                continue
            s1 = scores[index]
            s2 = bt.run(ema_period=Xnew[0], period=Xnew[1],
                       up_trend_buy_value=Xnew[2],up_trend_sell_value=Xnew[3],
                       down_trend_buy_value=Xnew[4],down_trend_sell_value=Xnew[5],
                      )     
            if math.isnan(s2['Sharpe Ratio']):
                continue
            if s2['Sharpe Ratio'] > s1:
                wolf[0] = Xnew[0]
                wolf[1] = Xnew[1]
                wolf[2] = Xnew[2]
                wolf[3] = Xnew[3]
                wolf[4] = Xnew[4]
                wolf[5] = Xnew[5]
                scores[index] = s2['Sharpe Ratio']
        print(f'Stock: {stock}, Iteration: {i}, Best Sharpe: {max(scores)}')
    best = population[np.where(scores == sorted(scores)[-1])[0][-1]]
    best_params.loc[stock] = {'Ema Period':int(round(best[0])), 'Period':int(round(best[1])), 'Up Trend Buy Value':best[2],
                            'Up Trend Sell Value':best[3],
                           'Down Trend Buy Value':best[4],'Down Trend Sell Value':best[5]}
    stats = bt.run(ema_period=best[0], period=best[1],
                   up_trend_buy_value=best[2],up_trend_sell_value=best[3],
                   down_trend_buy_value=best[4],down_trend_sell_value=best[5],
                  )
    train_stats.loc[stock] = {'Sharpe Ratio': stats['Sharpe Ratio'], 'Annual Return': stats['Return (Ann.) [%]']}

Stock: AMD, Iteration: 1, Best Sharpe: 0.646011809824135
Stock: AMD, Iteration: 2, Best Sharpe: 0.646011809824135
Stock: AMD, Iteration: 3, Best Sharpe: 0.646011809824135
Stock: AMD, Iteration: 4, Best Sharpe: 0.646011809824135
Stock: AMD, Iteration: 5, Best Sharpe: 0.646011809824135
Stock: AMD, Iteration: 6, Best Sharpe: 0.646011809824135
Stock: AMD, Iteration: 7, Best Sharpe: 0.646011809824135
Stock: AMD, Iteration: 8, Best Sharpe: 0.646011809824135
Stock: AMD, Iteration: 9, Best Sharpe: 0.646011809824135
Stock: AMD, Iteration: 10, Best Sharpe: 0.646011809824135
Stock: AMD, Iteration: 11, Best Sharpe: 0.646011809824135
Stock: AMD, Iteration: 12, Best Sharpe: 0.7014554223161674
Stock: AMD, Iteration: 13, Best Sharpe: 0.7014554223161674
Stock: AMD, Iteration: 14, Best Sharpe: 0.7014554223161674
Stock: AMD, Iteration: 15, Best Sharpe: 0.7014554223161674
Stock: AMD, Iteration: 16, Best Sharpe: 0.7014554223161674
Stock: AMD, Iteration: 17, Best Sharpe: 0.7014554223161674
Stock: AMD, Itera

In [24]:
best_params

,Ema Period,Period,Up Trend Buy Value,Up Trend Sell Value,Down Trend Buy Value,Down Trend Sell Value
AMD,69,26,41.731545,55.349008,17.942940,56.094657
AAPL,63,11,54.117314,55.508361,26.747182,37.488459
JPM,120,12,40.185428,55.372455,44.602205,68.178255
PEP,169,11,43.105284,50.936482,40.376330,85.888716
NFLX,65,12,55.602980,65.390608,14.752867,55.958529


In [25]:
train_stats.loc['Mean'] = {'Sharpe Ratio':train_stats['Sharpe Ratio'].mean(),'Annual Return':train_stats['Annual Return'].mean()}
train_stats

,Sharpe Ratio,Annual Return
AMD,1.006199,19.788835
AAPL,1.377578,33.446944
JPM,1.050210,22.622998
PEP,0.990626,16.440507
NFLX,0.976975,47.710948
Mean,1.080317,28.002047


In [26]:
for index,row in best_params.iterrows():
    bt = Backtest(tests[stocks.index(index)], RSITrendStrategy, cash=10_000, commission=0)
    stats = bt.run(ema_period = row['Ema Period'],period=row['Period'],
                   up_trend_buy_value=row['Up Trend Buy Value'],up_trend_sell_value=row['Up Trend Sell Value'],
                   down_trend_buy_value=row['Down Trend Buy Value'],down_trend_sell_value=row['Down Trend Sell Value'],
                  )
    test_stats.loc[index] = {'Sharpe Ratio': stats['Sharpe Ratio'], 'Annual Return': stats['Return (Ann.) [%]']}
test_stats.loc['Mean'] = {'Sharpe Ratio':test_stats['Sharpe Ratio'].mean(),'Annual Return':test_stats['Annual Return'].mean()}
test_stats

,Sharpe Ratio,Annual Return
AMD,0.000000,-4.378407
AAPL,1.084580,22.401648
JPM,1.239469,19.973044
PEP,1.342717,22.781237
NFLX,0.086820,2.402899
Mean,0.750717,12.636084
